Comparison between Google Cloud NetApp Volumes and Filestore
------------------------------------------------------------
These tests were done on a Medium (2CPU, 8GB) instance on data-dev.lsst.cloud, with the `w_2025_07` image, using the default NFSv3 mount parameters `rw,relatime,vers=3,rsize=1048576,wsize=1048576,namlen=255,hard,proto=tcp,timeo=600,retrans=2,sec=sys,mountvers=3,mountproto=udp,local_lock=none` and iozone 3.507 with `iozone -Rac -g 16G`.

In [ ]:
import json
import math
from pathlib import Path
from comparator import Comparator

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
c = Comparator()

In [ ]:
c.compare()

In [ ]:
cmp = json.loads((Path("comparison") / "netapp-to-filestore-ratio.json").read_text())

In [ ]:
def plot_ratio(inp_data, category: str, filesize:str):
    spec_data=inp_data[category][filesize]
    xdata = [ int(x) for x in spec_data.keys() ]
    ydata = [ x["ratio"] for x in spec_data.values() ]
    fig=px.line(x=xdata, y=ydata, log_x = True,
           labels = { "x": "blocksize in kB",
                      "y": "Netapp-to-filestore performance ratio" },
            title = f"{category} {filesize}kB file performance ratio"
           )
    fig.show()


In [ ]:
def plot_bar(inp_data, category: str, filesize:str):
    spec_data = inp_data[category][filesize]
    xdata = [ math.log(int(x),2) for x in spec_data.keys()]
    nadata = [ 1024 * x["netapp"] for x in spec_data.values() ]
    fsdata = [ 1024 * x["filestore"] for x in spec_data.values() ]
    fig = go.Figure(data=[
        go.Bar(name="netapp", x=xdata, y=nadata),
        go.Bar(name="filestore", x=xdata, y=fsdata)
    ])
    fig.update_layout(barmode="group",
                      title = f"{category} {filesize}kB file performance")
    fig.update_xaxes(title={"text": "log2(blocksize in kB)"})
    fig.update_yaxes(title={"text": "bytes/s"})
    fig.show()
    

In [ ]:
categories=("reader","writer")
# categories=("random-read", "random-write")
# categories=("stride-read", "record-rewrite")
for category in categories:
    for filesize in cmp[category].keys():
        # plot_ratio(cmp, category, filesize)
        plot_bar(cmp, category, filesize)